In [ ]:
import pathlib
import shutil
import os

import numpy as np
import pandas as pd

import dbfread

In [ ]:
original_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB')
decoded_dir = pathlib.Path(r'S:\DataExchange\SASH\iViewDB_decoded')

decoded_dir.mkdir(exist_ok=True)

In [ ]:
# table_ids['frame']

In [ ]:
for i, encoded_image_path in enumerate(images_to_be_indexed):
    frame_index = table_ids['frame'].index(file_image_ids[i])
    pimg_dbid = table_records['frame'][frame_index]['PIMG_DBID']
    image_data = table_records['patient_image'][table_ids['patient_image'].index(pimg_dbid)]
    port_dpid = image_data['PORT_DBID']

    image_date_time = f"{image_data['IMG_DATE']}_{image_data['IMG_TIME']}"

    machine = image_data['ORG_DTL']
    
    unique_index = pimg_id_unique.index(pimg_dbid)

    first_index = pimg_id_first_index[unique_index]
    total_frames = pimg_id_counts[unique_index]

    frame_number_str_length = len(str(total_frames))
    
    current_frame = frame_index - first_index
    current_frame_str = str(current_frame).zfill(frame_number_str_length)
    
    port_data = table_records['port'][table_ids['port'].index(port_dpid)]

    port_name = port_data['ID']
    treatment_dbid = port_data['TRT_DBID']
    
    treatment_data = table_records['treatment'][table_ids['treatment'].index(treatment_dbid)]

    treatment_name = treatment_data['ID']
    patient_dbid = treatment_data['PAT_DBID']
    
    patient_data = table_records['patient'][table_ids['patient'].index(patient_dbid)]
    patient_dir = f"{patient_data['ID']}_{patient_data['LAST_NAME']}_{patient_data['FIRST_NAME']}"
    
    save_dir = indexed_dir.joinpath(machine, patient_dir, treatment_name, port_name, image_date_time)
    save_dir.mkdir(parents=True, exist_ok=True)
    
    new_path = save_dir.joinpath(f'frame_{current_frame_str}.png')
    
    shutil.copy2(encoded_image_path, home.joinpath('temp_encoded.jpg'))
    
    !bash -c "~/bin/wavelet/jpeg /mnt/c/Users/sbiggs/temp_encoded.jpg /mnt/c/Users/sbiggs/temp_decoded.jpg"
    !bash -c "convert -depth 16 /mnt/c/Users/sbiggs/temp_decoded.jpg /mnt/c/Users/sbiggs/temp_decoded.png"
    
    shutil.copy2(home.joinpath('temp_decoded.png'), new_path)
    encoded_image_path.unlink()